<a href="https://colab.research.google.com/github/Annalina-Luo/ClipNews/blob/main/ClipNews_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Process notenook for [ClipNews]((https://github.com/Annalina-Luo/NewsClip.git))

In [1]:
#@title Install
# you can upload the codes or clone them from github.
!git clone https://github.com/Annalina-Luo/ClipNews.git
!pip install transformers
!pip install git+https://github.com/openai/CLIP.git

Cloning into 'ClipNews'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 224 (delta 117), reused 119 (delta 42), pack-reused 0
Receiving objects: 100% (224/224), 19.62 MiB | 28.41 MiB/s, done.
Resolving deltas: 100% (117/117), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-os9eiqnt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-os9eiqnt
  Resolved https://github.c

In [2]:
cd ClipNews/

/content/ClipNews


In [3]:
#@title Download the training data and checkpoints for previous models
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

class Downloader(object):
    def __init__(self):
        self.authenticate()
        
    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)
    
    def download_file(self, file_id, file_dst):
        downloaded = self.drive.CreateFile({'id':file_id})
        downloaded.FetchMetadata(fetch_all=True)
        downloaded.GetContentFile(file_dst)

downloader = Downloader()

In [4]:
# it will take a long time to download our data
downloader.download_file("1XQ1iAFs91jOtP6DiklAxnd3Mq7ZPGIrx", "images_processed.tar.gz")
downloader.download_file("1MG8svnIq_h8_WpNOcCVM86t_sKGrsykR", "train.json")
downloader.download_file("1h8G_dOyK6eMEFmTIkX4ofdvDV1GjZ2nv", "val.json")
downloader.download_file("1H22CjcLBJI5UB1zsw6j1jRTkouq11Rds", "val_gts.json")
downloader.download_file("1pzuxjyMmvFPS_qk2Lfro-PkDbC6gTqj5", "./checkpoint/checkpoint_ClipNews.pth.tar")

In [5]:
# it will take a long time to unzip the image files
!tar -zxf ./images_processed.tar.gz

In [ ]:
#@title Training Process
# change the path of annotation files using `--ann_path`
# change the path of gts file files using `--gts_file_dev`
# change the attention module in the architecture `--TextEncoder_attention=True` or `--ImageEncoder_attention=True`
# you can continue our last training using `--checkpoint="./checkpoint/checkpoint_ClipNews.pth.tar"` (using GPU)
!python main.py --num_workers=0 --image_dir="./images_processed/" --batch_size=64 --TextEncoder_attention=True

Train 28:   0% 0/170 [00:00<?, ?it/s]/content/NewsClip/model.py:193: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = torch.tensor(out, dtype=torch.float32)
/content/NewsClip/model.py:392: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  index1 = torch.tensor(index1, dtype=torch.int64)
Train 28: 100% 170/170 [1:13:37<00:00, 25.99s/it]
Epoch [28/150], Loss: 0.0636, Perplexity: 1.0657
Train 29: 100% 170/170 [1:12:50<00:00, 25.71s/it]
Epoch [29/150], Loss: 0.0275, Perplexity: 1.0279
Dev 29:   0% 0/10870 [00:00<?, ?it/s]2023-04-12 20:00:37.176844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Dev 29:   0% 1/10870 [00:08<26:53:19,  8.91s/it]/c